# Amazon SageMaker - Bring Your Own Container
## TensorFlow 2.0 編
ここでは TensorFlow2.0 を使ったサンプルコードを題材に、Amazon SageMaker で独自コンテナを用いた機械学習モデルの学習方法を順を追って説明します。トレーニングスクリプトは既に SageMaker 向けに書き換えられた形で準備され、その上で、独自の学習用コンテナを活用します。 トレーニングスクリプトを SageMaker 向けに書き換える際は  [Bring Your Own Model ハンズオンワークショップ](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/workshop/lab_bring-your-own-model) をご参考に下さい。

In [ ]:
import os
from datetime import datetime
import keras
import numpy as np
from keras.datasets import mnist

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()
bucket = sess.default_bucket()

## 2. Notebook 上でのデータ準備

トレーニングを始める前に、予めこの Notebook にデータを準備します。今回は [Keras](https://keras.io/) の MNIST データを活用します。

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)

## 3. ローカルでの動作確認
トレーニングジョブを始める前に、この Notebook インスタンス上でコンテナを立て、学習がうまくいくか動作確認しましょう。

まずは今回学習につかう Dockerfile の中身を確認します。

In [ ]:
!cat container/Dockerfile

この Notebook インスタンス上の Docker へ Dockerfile を使ってイメージをビルドします。

In [ ]:
!docker build -t tf-2.0 container/.

`image_name` で今回ビルドした `tf-2.0` を指定し、`train_instance_type` を`local` にすることで、独自コンテナを使ってこの Notebook 上での学習をすることができます。Dockerfile やトレーニングスクリプトが適切に記述されているか確認しましょう。

In [ ]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()

estimator = Estimator(image_name='tf-2.0',
                      role=role,
                      hyperparameters={'batch_size': 64,'epochs': 1},
                      train_instance_count=1,
                      train_instance_type='local')

estimator.fit({'train': 'file://data'})

## 3. コンテナのビルドと Amazon ECR への登録
次に学習インスタンスで独自コンテナを活用するために、 今回ビルドしたイメージを Amazon ECR へ登録します。

In [ ]:
!bash container/build_and_push.sh

登録できているかどうか [ECR のコンソール](https://console.aws.amazon.com/ecr) で確認してみましょう。

学習ジョブにおいてこのコンテナを使うためにイメージパスが必要なので取得しましょう。

In [ ]:
client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'sagemaker-tf2.0-example'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print('使用する Docker Image のURIは {} です。'.format(ecr_image))

## 4. データの S3 へのアップロード
学習ジョブでトレーニングを始めるために、Amazon S3 にデータを準備しておく必要があります。

In [ ]:
bucket_name = sagemaker_session.default_bucket()
input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='dataset/mnist')

print('学習データは {} へアップロードされました。'.format(input_data))

## 5. トレーニングジョブの発行

In [ ]:
job_name='tf20-example-{0:%Y%m%d-%H%M%S}'.format(datetime.now())


estimator = Estimator(image_name=ecr_image,
                      role=role,
                      hyperparameters={'batch_size': 64,'epochs': 1},
                      train_instance_count=1,
                      train_instance_type='ml.p2.xlarge')

estimator.fit({'train': '{}'.format(input_data)}, job_name=job_name)

## 6. Amazon S3 へ保存されているモデルの確認
学習されたモデルは S3 へ保存されています。

In [ ]:
# モデルファイルとアウトプットファイルのダウンロード
s3 = boto3.client('s3')
s3.download_file(Bucket=bucket, Key= job_name + '/output/model.tar.gz', Filename = 'model.tar.gz')

# モデルの解凍
!tar -zxvf model.tar.gz

## 7. まとめ

TensorFlow 2.0 の Docker image を使った独自コンテナでの学習についてご紹介しました。 Dockerfile を編集することで様々なコンテナをお使い頂けますのでご活用下さい。